<a href="https://colab.research.google.com/github/Joriswillems/deeplearning/blob/master/assignment3/Assignment_3_1_82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3.1. Sequence Classification

# Task: Aspect-level Sentiment Classification(10pt)

<b>Group 82</b>
* <b> Student 1 </b> : Joris Willems + 0908753
* <b> Student 2 </b> : Lars Schilders + 0908729

Reading material:
- [1] R. He, WS. Lee & D. Dahlmeier. Exploiting document knowledge for aspect-level sentiment classification. 2018. https://arxiv.org/abs/1806.04346.


Build an attention-based aspect-level sentiment classification model with biLSTM. Your model shall include:

- BiLSTM network that learns sentence representation from input sequences.
- Attention network that assigns attention score over a sequence of biLSTM hidden states based on aspect terms representation.
- Fully connected network that predicts sentiment label, given the representation weighted by the attention score.

Requirements:

- You shall train your model bsaed on transferring learning. That is, you need first train your model on documnet-level examples. Then the learned weights will be used to initialize aspect-level model and fine tune it on aspect-level examples.
- You shall use the alignment score function in attention network as following expression:$$f_{score}(h,t)=tanh(h^TW_a t)$$
- You shall evaluate the trained model on the provided test set and show the accuracy on test set.


In [16]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [0]:
import os
import sys
import codecs
import operator
import numpy as np
import re
from time import time

In [0]:
import _pickle as cPickle


# Load Data

In [0]:
def read_pickle(data_path, file_name):

    f = open(os.path.join(data_path, file_name), 'rb')
    read_file = cPickle.load(f)
    f.close()

    return read_file

def save_pickle(data_path, file_name, data):

    f = open(os.path.join(data_path, file_name), 'wb')
    cPickle.dump(data, f)
    print(" file saved to: %s"%(os.path.join(data_path, file_name)))
    f.close()

In [0]:
aspect_path = '/drive/My Drive/Colab Notebooks/2IMM10 - Deep Learning/Assignment 3/aspect-level' 
# aspect_path = '/content/drive/My Drive/deeplearning/data/aspect-level/'
# aspect_path = '../courseFiles/Practicals/Practical 5/data/aspect-level/'

vocab = read_pickle(aspect_path, 'all_vocab.pkl')

train_x = read_pickle(aspect_path, 'train_x.pkl')
train_y = read_pickle(aspect_path, 'train_y.pkl')
dev_x = read_pickle(aspect_path, 'dev_x.pkl')
dev_y = read_pickle(aspect_path, 'dev_y.pkl')
test_x = read_pickle(aspect_path, 'test_x.pkl')
test_y = read_pickle(aspect_path, 'test_y.pkl')

train_aspect = read_pickle(aspect_path, 'train_aspect.pkl')
dev_aspect = read_pickle(aspect_path, 'dev_aspect.pkl')
test_aspect = read_pickle(aspect_path, 'test_aspect.pkl')


pretrain_data = read_pickle(aspect_path, 'pretrain_data.pkl')
pretrain_label = read_pickle(aspect_path, 'pretrain_label.pkl')

In [0]:
def DecodeSequence(idx):
    review = str([list(vocab.keys())[i] for i in train_x[idx] if i!=0])
    aspectTerm = [list(vocab.keys())[i] for i in train_aspect[idx]]
    target = ["Positive", "Negative", "Neutral"][int(np.nonzero(train_y[idx])[0])] 
    print("Full review: {}".format(review))
    print("Aspect terms: {}".format(aspectTerm))
    print("Target: {}".format(target))

In [0]:
class Dataiterator_doc():
    '''
      1) Iteration over minibatches using next(); call reset() between epochs to randomly shuffle the data
      2) Access to the entire dataset using all()
    '''
    
    def __init__(self, X, y, seq_length=32, decoder_dim=300, batch_size=32):      
        self.X = X 
        self.y = y 
        self.num_data = len(X) # total number of examples
        self.batch_size = batch_size # batch size
        self.reset() # initial: shuffling examples and set index to 0
    
    def __iter__(self): # iterates data
        return self


    def reset(self): # initials
        self.idx = 0
        self.order = np.random.permutation(self.num_data) # shuffling examples by providing randomized ids 
        
    def __next__(self): # return model inputs - outputs per batch
        X_ids = [] # hold ids per batch 
        while len(X_ids) < self.batch_size:
            X_id = self.order[self.idx] # copy random id from initial shuffling
            X_ids.append(X_id)
            self.idx += 1 # 
            if self.idx >= self.num_data: # exception if all examples of data have been seen (iterated)
                self.reset()
                raise StopIteration()
                
        batch_X = self.X[np.array(X_ids)] # X values (encoder input) per batch
        batch_y = self.y[np.array(X_ids)] # y_in values (decoder input) per batch
        return batch_X, batch_y

          
    def all(self): # return all data examples
        return self.X, self.y
class Dataiterator_aspect():
    '''
      1) Iteration over minibatches using next(); call reset() between epochs to randomly shuffle the data
      2) Access to the entire dataset using all()
    '''
    
    def __init__(self, aspect_data, seq_length=32, decoder_dim=300, batch_size=32):
        
        len_aspect_data = len(aspect_data[0])
        #self.len_doc_data = len(doc_data[0])
        
        self.X_aspect = aspect_data[0] 
        self.y_aspect = aspect_data[1]
        self.aspect_terms = aspect_data[2]  
        self.num_data = len_aspect_data
        self.batch_size = batch_size # batch size
        self.reset() # initial: shuffling examples and set index to 0
    
    def __iter__(self): # iterates data
        return self


    def reset(self): # initials
        self.idx = 0
        self.order = np.random.permutation(self.num_data) # shuffling examples by providing randomized ids 
        
    def __next__(self): # return model inputs - outputs per batch
        
        X_ids = [] # hold ids per batch 
        while len(X_ids) < self.batch_size:
            X_id = self.order[self.idx] # copy random id from initial shuffling
            X_ids.append(X_id)
            self.idx += 1 # 
            if self.idx >= self.num_data: # exception if all examples of data have been seen (iterated)
                self.reset()
                raise StopIteration()
                
        batch_X_aspect = self.X_aspect[np.array(X_ids)] # X values (encoder input) per batch
        batch_y_aspect = self.y_aspect[np.array(X_ids)] # y_in values (decoder input) per batch
        batch_aspect_terms = self.aspect_terms[np.array(X_ids)]
        
        return batch_X_aspect, batch_y_aspect, batch_aspect_terms

          
    def all(self): # return all data examples
        return self.X_aspect, self.y_aspect, self.aspect_terms

In [22]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Lambda, Dropout, LSTM,Bidirectional
from keras.layers import Reshape, Activation, RepeatVector, concatenate, Concatenate, Dot, Multiply, Add
import keras.backend as K
from keras.engine.topology import Layer
from keras import initializers
from keras import regularizers
from keras import constraints


Using TensorFlow backend.


In [0]:
overal_maxlen = 82
overal_maxlen_aspect = 7


# Define Attention Network Layer
- Define class for Attention Layer
- You need to finish the code for calculating the attention weights

In [0]:

class Attention(Layer):
    def __init__(self,  **kwargs):
        """
        Keras Layer that implements an Content Attention mechanism.
        Supports Masking.
        """
       
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert type(input_shape) == list
       
        self.steps = input_shape[0][1]

        self.W = self.add_weight(shape=(input_shape[0][-1], input_shape[1][-1]),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),)

        self.built = True

    def compute_mask(self, input_tensor, mask=None):
        assert type(input_tensor) == list
        assert type(mask) == list
        return None

    def call(self, input_tensor, mask=None):
        
        # output of BiLSTM for sentence (h in paper)
        x = input_tensor[0]  #(None, 82, 600)
        
        # output of word embedding layer of aspect terms (t in paper)
        aspect = input_tensor[1] #(None, 300)
        
        # used to remove influence of padded value
        #mask = mask[0]
        
        #(None, 600)
        aspect = K.transpose(K.dot(self.W, K.transpose(aspect)))
        
        #(None, 1, 600)
        aspect = K.expand_dims(aspect, axis=-2)
        
        #(None, 82, 600)
        aspect = K.repeat_elements(aspect, self.steps, axis=1)
        
        #(None, 82)
        beta_vec = K.tanh(K.sum(x*aspect, axis=-1))
        
        #(None, 82)
        alpha = K.exp(beta_vec)
        alpha /= K.cast(K.sum(alpha, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        
        return alpha
    
   
    def compute_output_shape(self, input_shape):
        print((input_shape[0][0], input_shape[0][1]))
        return (input_shape[0][0], input_shape[0][1]) #(None, 82)


    ##### van paper ######
    def call2(self, input_tensor, mask=None):
        x = input_tensor[0]
        aspect = input_tensor[1]
        mask = mask[0]

        aspect = K.transpose(K.dot(self.W, K.transpose(aspect)))
        aspect = K.expand_dims(aspect, axis=-2)
        aspect = K.repeat_elements(aspect, self.steps, axis=1)
        eij = K.sum(x*aspect, axis=-1)

        if self.bias:
            b = K.repeat_elements(self.b, self.steps, axis=0)
            eij += b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        return a


In [0]:
class Average(Layer):
  
    def __init__(self, mask_zero=True, **kwargs):
        self.mask_zero = mask_zero
        self.supports_masking = True
        super(Average, self).__init__(**kwargs)

    def call(self, x,mask=None):
        if self.mask_zero:           
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask)
            x = x * mask
            return K.sum(x, axis=1) / (K.sum(mask, axis=1) + K.epsilon())
        else:
            return K.mean(x, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
    def compute_mask(self, x, mask):
        return None

# Establish computation Grah for model
- Input tensors
- Shared WordEmbedding layer 
- Attention network layer  
- Shared BiLSTM layer
- Shared fully connected layer(prediction layer)


In [0]:
dropout = 0.5     
recurrent_dropout = 0.1
vocab_size = len(vocab)
num_outputs = 3 # labels

## Input tensors

In [0]:
#pretrain data: document-level input (30000,300), output (30000,3)
#[train_x, aspect_train]: aspect_level input (n_samples, 82) en (n_samples,7)

In [27]:
#YOUR CODE HERE ##### Inputs #####
sentence_input = Input(shape=(overal_maxlen,), dtype='int32', name='sentence_input')
aspect_input = Input(shape=(overal_maxlen_aspect,), dtype='int32', name='aspect_input')
pretrain_input = Input(shape=(None,), dtype='int32', name='pretrain_input')


print(sentence_input)
print(aspect_input)
print(pretrain_input)

Tensor("sentence_input:0", shape=(None, 82), dtype=int32)
Tensor("aspect_input:0", shape=(None, 7), dtype=int32)
Tensor("pretrain_input:0", shape=(None, None), dtype=int32)


## Shared WordEmbedding layer

In [28]:
#YOUR CODE HERE### represent aspect as averaged word embedding ###
word_emb = Embedding(vocab_size, 300, mask_zero=True, name='word_emb')
aspect_term_embs = word_emb(aspect_input)
aspect_embs = Average(mask_zero=True, name='aspect_emb')(aspect_term_embs)

print(aspect_term_embs)
print(aspect_embs)

Tensor("word_emb/embedding_lookup/Identity_1:0", shape=(None, 7, 300), dtype=float32)
Tensor("aspect_emb/truediv:0", shape=(None, 300), dtype=float32)


In [29]:
#YOUR CODE HERE ### sentence representation from embedding ###
sentence_embs = word_emb(sentence_input) # from aspect-level domain
pretrain_embs = word_emb(pretrain_input) # from document-level domain

print(sentence_embs)
print(pretrain_embs)

Tensor("word_emb_1/embedding_lookup/Identity_1:0", shape=(None, 82, 300), dtype=float32)
Tensor("word_emb_2/embedding_lookup/Identity_1:0", shape=(None, None, 300), dtype=float32)


## Shared BiLSTM layer

In [30]:
#YOUR CODE HERE ### sentence representation from embedding ###
bilstm = Bidirectional(LSTM(300, return_sequences=True, dropout=dropout, recurrent_dropout=recurrent_dropout), name='BiLSTM')
pretrain_lstm = bilstm(pretrain_embs) #pretrain the bilstm (no attention involved) MUST BE AVERAGED
sentence_lstm = bilstm(sentence_embs) #finetune bilstm with attention mechanism to obtain aspect-level info

sentence_lstm

<tf.Tensor 'BiLSTM_1/concat:0' shape=(None, 82, 600) dtype=float32>

In [31]:
pretrain_lstm

<tf.Tensor 'BiLSTM/concat:0' shape=(None, None, 600) dtype=float32>

## Attention Layer

In [32]:
##YOUR CODE HERE
attention = Attention()([sentence_lstm, aspect_embs])
attention

(None, 82)


<tf.Tensor 'attention_1/truediv:0' shape=(None, 82) dtype=float32>

## Prediction Layer

In [33]:
z = Dot(axes=1)([sentence_lstm, attention])

predictionLayer = Dense(3, activation='softmax')

aspect_probs = predictionLayer(z)

pretrain_avg = Average(mask_zero=True)(pretrain_lstm)
pretrain_probs = predictionLayer(pretrain_avg)

pretrain_probs

<tf.Tensor 'dense_1_1/Softmax:0' shape=(None, 3) dtype=float32>

In [34]:
aspect_probs

<tf.Tensor 'dense_1/Softmax:0' shape=(None, 3) dtype=float32>

# Build Models for document-level and aspect-level data
- The two models shared the embedding, BiLSTM, Prediction Layer

In [35]:
### YOUR CODE HERE
model1 = Model(inputs=[pretrain_input], outputs=[pretrain_probs])

model2 = Model(inputs=[sentence_input, aspect_input], outputs=[aspect_probs])

model2.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sentence_input (InputLayer)     (None, 82)           0                                            
__________________________________________________________________________________________________
aspect_input (InputLayer)       (None, 7)            0                                            
__________________________________________________________________________________________________
word_emb (Embedding)            multiple             3000900     aspect_input[0][0]               
                                                                 sentence_input[0][0]             
__________________________________________________________________________________________________
BiLSTM (Bidirectional)          multiple             1442400     word_emb[1][0]             

# Train Model
- First Train model on document-level data.
- Then Train  model on aspect-level data

## Train on document-level data

In [0]:

import keras.optimizers as opt
optimizer=opt.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06, clipnorm=10, clipvalue=0)
model1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
batch_size = 128
train_steps_epoch = len(pretrain_data)/batch_size
batch_train_iter_doc = Dataiterator_doc(pretrain_data, pretrain_label, batch_size=batch_size)



In [42]:
###YOUR CODE HERE###

from keras.callbacks import EarlyStopping


history1 = model1.fit_generator(batch_train_iter_doc, 
                     steps_per_epoch=train_steps_epoch,
                     epochs=20, 
                     )

Epoch 1/20
 67/234 [=======>......................] - ETA: 5:35 - loss: 0.7749 - categorical_accuracy: 0.6500

KeyboardInterrupt: ignored

## Train on aspect-level data

In [0]:
train_steps_epoch = len(train_x)/batch_size
batch_train_iter_aspect = Dataiterator_aspect([train_x, train_y, train_aspect], batch_size=batch_size)
val_steps_epoch = len(dev_x)/batch_size
batch_val_iter_aspect = Dataiterator_aspect([dev_x, dev_y, dev_aspect], batch_size=batch_size)

import keras.optimizers as opt
optimizer = opt.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=10, clipvalue=0)
model2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])


In [45]:
### YOUR CODE HERE


def train_gen_aspect():
    while True:
        train_batches = [[[X, aspect], [y]] for X, y, aspect in batch_train_iter_aspect]
        
        for batch in train_batches:
            yield batch
            
def val_gen_aspect():
    while True:
        val_batches = [[[X, aspect], [y]] for X, y, aspect in batch_val_iter_aspect]
        
        for batch in val_batches:
            yield batch

es = EarlyStopping(monitor='val_loss', patience=2)
  
history2 = model2.fit_generator(train_gen_aspect(), 
                     steps_per_epoch=train_steps_epoch, 
                     epochs=10,
                     validation_data=val_gen_aspect(),
                     validation_steps=val_steps_epoch,
                     callbacks=[es]
                     )

Epoch 1/10
15/14 [===============================] - 12s 795ms/step - loss: 0.8999 - categorical_accuracy: 0.5844 - val_loss: 0.9135 - val_categorical_accuracy: 0.5918
Epoch 2/10
15/14 [===============================] - 10s 692ms/step - loss: 0.8342 - categorical_accuracy: 0.6245 - val_loss: 0.8012 - val_categorical_accuracy: 0.6172
Epoch 3/10
15/14 [===============================] - 10s 698ms/step - loss: 0.8182 - categorical_accuracy: 0.6438 - val_loss: 0.8430 - val_categorical_accuracy: 0.6562
Epoch 4/10
15/14 [===============================] - 11s 704ms/step - loss: 0.7897 - categorical_accuracy: 0.6724 - val_loss: 0.8374 - val_categorical_accuracy: 0.6523


## Evaluating on test set
- show the accuracy

In [0]:
##YOUR CODE HERE

In [49]:
loss, accuracy = model2.evaluate([test_x, test_aspect], test_y)

print("Categorical accuracy: {:.3f}%".format(accuracy*100))

638/638 [==============================] - 2s 3ms/step
Categorical accuracy: 57.994%
